
Imports e definição do dataset.

In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [52]:
# Carregar o conjunto de dados CIFAR-10
#Carrega duas tuplas, representando os dados de treinamento e de teste.
#Cada tupla tem as imagens e os respectivos rótulos
datasets = {
    'cifar10'       : [keras.datasets.cifar10,       (32, 32, 3),  10],
    'cifar100'      : [keras.datasets.cifar100,      (32, 32, 3), 100],
    'mnist'         : [keras.datasets.mnist,         (28, 28, 1),  10],
    'fashion_mnist' : [keras.datasets.fashion_mnist, (28, 28, 1),  10]
}

dataset = datasets['fashion_mnist']

Montar as redes que vão ser usadas para testar.

Todas as redes compartilham a mesma "head" densa, uma camada  com 64 neurônios e ativação ReLU e uma camada de saída com n (onde n é o número de classes do dataset) neurônios e ativação softmax.

In [53]:
# Layer Residual (Implementação baseada em: https://gist.github.com/FirefoxMetzger/6b6ccf4f7c344459507e73bbd13ec541)
class Residual(tf.keras.layers.Layer):
    def __init__(self, channels_in,kernel, strides=1, **kwargs):
        super(Residual, self).__init__(**kwargs)
        self.channels_in = channels_in
        self.kernel = kernel
        self.conv1 = tf.keras.layers.Conv2D(self.channels_in,
                                self.kernel,
                                padding = 'same',
                                strides=strides,
                                activation='relu')
        self.batch1 = tf.keras.layers.BatchNormalization()

        self.conv2 = tf.keras.layers.Conv2D(self.channels_in,
                                self.kernel,
                                padding = 'same',
                                strides=strides,
                                activation='relu')

        self.batch2 = tf.keras.layers.BatchNormalization()
        self.conv3 = tf.keras.layers.Conv2D(self.channels_in,
                                self.kernel,
                                padding = 'same',
                                strides=strides,
                                activation='relu')
        self.batch3 = tf.keras.layers.BatchNormalization()

        self.conv4 = tf.keras.layers.Conv2D(self.channels_in,
                                self.kernel,
                                padding = 'same',
                                strides=strides,
                                activation='relu')

    def call(self, x):
        # the residual block using Keras functional API
        input_layer =   tf.keras.layers.Activation('linear')(x)
        x =             self.conv1(input_layer)
        x =             self.batch1(x)
        x =             tf.keras.layers.Dropout(0.25)(x)
        x =             self.conv2(x)
        x =             self.batch2(x)
        x =             tf.keras.layers.Dropout(0.25)(x)
        x =             self.conv3(x)
        x =             self.batch3(x)
        x =             self.conv3(x)
        residual =      tf.keras.layers.Add()([x, input_layer])

        return x

    def compute_output_shape(self, input_shape):
        return input_shape

In [59]:
# Crie o modelo de rede neural convolucional
input_shape = dataset[1]
def get_advanced_network():
  model =  keras.Sequential([
      tf.keras.layers.Conv2D(64, (2, 2), strides=2, activation='relu', input_shape=input_shape),
      Residual(64, (7,7), strides=1),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dropout(0.25),
      tf.keras.layers.Conv2D(128, (5, 5), strides=1, activation='relu'),
      tf.keras.layers.MaxPool2D(pool_size=(2,2)),

      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dropout(0.25),

      tf.keras.layers.Conv2D(128, (3, 3), strides=1, activation='relu'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.GlobalAveragePooling2D()
  ])
  return model

def compose_with_simple_head(backbone, num_classes):
    model = tf.keras.models.clone_model(backbone)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu')),
    tf.keras.layers.Dropout(0.1),
    model.add(tf.keras.layers.Dense(128, activation='relu')),
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

    # Compile o modelo
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',#pode ser substituída pela esparse_categorical_cross_entropy
                  metrics=['accuracy'])

def compose_with_advanced_head(backbone, num_classes):
    model = tf.keras.models.clone_model(backbone)
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

    # Compile o modelo
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',#pode ser substituída pela esparse_categorical_cross_entropy
                  metrics=['accuracy'])

    model.summary()

    return model

Trecho para treinar e avaliar a rede neural.
O treino é realizado com os dados de treino e a avaliação do modelo é realizada nos dados de teste.

In [60]:
(train_images, train_labels), (test_images, test_labels) = dataset[0].load_data()
num_classes = dataset[2]

# Convert the labels to one-hot encoding
train_labels = keras.utils.to_categorical(train_labels, num_classes)
test_labels = keras.utils.to_categorical(test_labels, num_classes)

# Treine o modelo
model = compose_with_advanced_head(get_advanced_network(), num_classes)
model.fit(train_images, train_labels, epochs=10)

# Avalie o modelo no conjunto de teste
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Acurácia no conjunto de teste: {test_accuracy * 100:.2f}%')

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_345 (Conv2D)         (None, 14, 14, 64)        320       
                                                                 
 residual_37 (Residual)      (None, 14, 14, 64)        603072    
                                                                 
 batch_normalization_282 (B  (None, 14, 14, 64)        256       
 atchNormalization)                                              
                                                                 
 dropout_59 (Dropout)        (None, 14, 14, 64)        0         
                                                                 
 conv2d_350 (Conv2D)         (None, 10, 10, 128)       204928    
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 5, 5, 128)         0         
 g2D)                                                

Na célula abaixo, adicione o código para carregar os demais datasets

Tarefa
Escreva código para executar redes neurais nos seguintes datasets:

MNIST (pode aproveitar o codigo existente)
Fashion MNIST
CIFAR-10
CIFAR-100
Cada execução deve ser por 10 épocas.

Você deve preencher as funções a seguir para retornarem a rede neural com a melhor configuração que você conseguiu para cada dataset. O notebook deve ser entregue com a rede neural que obteve a melhor performance em cada conjunto de dados.

IMPORTANTE: as funções não devem TREINAR nem AVALIAR as redes neurais, apenas instanciá-las e retorná-las.

Ao final, preencha o dict results com o desempenho encontrado em cada execução.

In [ ]:

def get_fashion_mnist_backbone():
  model =  keras.Sequential([
      tf.keras.layers.Conv2D(64, (2, 2), strides=2, activation='relu', input_shape=datasets['fashion_mnist'][1]),
      Residual(64, (7,7), strides=1),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dropout(0.25),
      tf.keras.layers.Conv2D(128, (5, 5), strides=1, activation='relu'),
      tf.keras.layers.MaxPool2D(pool_size=(2,2)),

      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dropout(0.25),

      tf.keras.layers.Conv2D(128, (3, 3), strides=1, activation='relu'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.GlobalAveragePooling2D()
  ])
  return model

def get_fashion_mnist_network():
  return compose_with_simple_head(get_fashion_mnist_backbone(), datasets['fashion_mniist'][2])

def get_mnist_backbone():
  model =  keras.Sequential([
    tf.keras.layers.Conv2D(32, (2, 2), strides=2, activation='relu', input_shape=datasets['mnist'][1]),
    Residual(32, (7,7)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (5, 5), strides=1, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(128, (3, 3), strides=1, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    Residual(128, (3,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(256, (2, 2), strides=1, activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D()
  ])
  return model

def get_mnist_network():
  return compose_with_simple_head(get_mnist_backbone(), datasets['mnist'][2])

def get_cifar100_backbone():
  model =  keras.Sequential([
      tf.keras.layers.Conv2D(64, (2, 2), strides=1, activation='relu', input_shape=input_shape),
      Residual(64, (7,7), strides=1),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dropout(0.25),
      tf.keras.layers.Conv2D(128, (5, 5), strides=1, activation='relu'),
      tf.keras.layers.MaxPool2D(pool_size=(2,2)),

      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dropout(0.25),

      tf.keras.layers.Conv2D(128, (3, 3), strides=1, activation='relu'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.GlobalAveragePooling2D()
  ])
  return model

def get_cifar100_network():
    return compose_with_simple_head(get_cifar100_backbone(), datasets['cifar100'][2])

def get_cifar10_backbone():
  model =  keras.Sequential([
      tf.keras.layers.Conv2D(128, (2, 2), strides=2, activation='relu', input_shape=datasets['cifar10'][1]),
      Residual(128, (7,7)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dropout(0.25),
      tf.keras.layers.Conv2D(64, (5, 5), strides=1, activation='relu'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dropout(0.25),

      tf.keras.layers.Conv2D(128, (3, 3), strides=1, activation='relu'),
      tf.keras.layers.BatchNormalization(),

      Residual(128, (3,3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dropout(0.25),
      tf.keras.layers.Conv2D(256, (2, 2), strides=1, activation='relu'),
      tf.keras.layers.GlobalAveragePooling2D()
  ])
  return model

def get_cifar10_network():
  return compose_with_simple_head(get_cifar10_backbone(), datasets['cifar10'][2])

Preencha o dict abaixo substituindo os None com a acuracia final (acc) e o tempo de treinamento (time) encontrado no seu experimento pra cada dataset

In [ ]:
results = {
    "mnist": {"time": 181, "acc": 0.9930},
    "fashion_mnist": {"time": 244, "acc": 0.9168},
    "cifar10": {"time": 545, "acc": 0.7495},
    "cifar100": {"time": 600, "acc": 0.3581},
}

Modelo "Caro".

**NÃO TENTAR TREINAR NO COLLAB NORMAL, O GOOGLE VAI BLOQUEAR OS RECURSOS.**

In [ ]:

def get_expensive_network():
  model =  keras.Sequential([
      tf.keras.layers.Conv2D(128, (1, 1), strides=1, activation='relu', input_shape=input_shape),
      Residual(128, (7,7), strides=1),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dropout(0.25),
      tf.keras.layers.Conv2D(128, (5, 5), strides=1, activation='relu'),
      #tf.keras.layers.MaxPool2D(pool_size=(2,2)),

      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dropout(0.25),

      tf.keras.layers.Conv2D(128, (3, 3), strides=1, activation='relu'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.GlobalAveragePooling2D()
  ])
  return model